#### dataset

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/13578/588368/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1570368151&Signature=X4kCxegOxLlIL9yWxP6yokUvwifvIs9m1uotdBpOBzSrOscYBRTxPlGMgRJDrxI6U6UqP4Q%2BIX2v7W1%2B0cH%2FDlnsXK5NSySxPY41SGTKf%2BeoumVJwsxOFi5ixAjxjNkQ5wumoqLYS58H%2FP4FOAmPCRYY6K5fKghrjxB6cHFBCTHLD3dCAyq8eWFdTUp8YiKK1NoHZWD1Ml3Hs2I6oRzDGcEX7DrAwINX0dONBSFgnFZ%2F%2B6ZFQut8Xpo8zdT%2BFlKekYyjJ9t2%2FIeq5FCcd%2B0HMwLALulY8wI39XM9UIaTm63BIykj5VAx3N%2FZPTLEy0kuQtrj0gXY%2FVwi7R9k1CvYrw%3D%3D&response-content-disposition=attachment%3B+filename%3Dkuzushiji-recognition.zip" -O "kuzushiji-recognition.zip" -c

In [0]:
!unzip -q kuzushiji-recognition.zip
!rm kuzushiji-recognition.zip

In [0]:
!mkdir train_images
!unzip -q train_images.zip -d train_images
!rm train_images.zip

In [0]:
!mkdir test_images
!unzip -q test_images.zip -d test_images
!rm test_images.zip

In [0]:
!ls

In [0]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

#### sub

In [95]:
import pandas as pd
df1 = pd.read_csv('train.csv')
df1.head()

,image_id,labels
0,100241706_00004_2,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...
1,100241706_00005_1,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...
2,100241706_00005_2,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...
3,100241706_00006_1,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...
4,100241706_00007_2,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...


In [96]:
df1.labels[0]

'U+306F 1231 3465 133 53 U+304C 275 1652 84 69 U+3044 1495 1218 143 69 U+3051 220 3331 53 91 U+306B 911 1452 61 92 U+306B 927 3445 71 92 U+306E 904 2879 95 92 U+5DE5 1168 1396 187 95 U+3053 289 3166 69 97 U+4E09 897 3034 121 107 U+306E 547 1912 141 108 U+3084 1489 2675 151 109 U+3068 1561 2979 55 116 U+5DF1 1513 2500 127 117 U+3082 1213 1523 72 119 U+3055 1219 3266 95 124 U+306E 259 2230 68 125 U+306E 1184 2423 169 125 U+4E16 849 2236 163 127 U+7D30 1144 1212 200 128 U+305D 316 3287 57 133 U+4EBA 217 2044 183 135 U+3051 277 2974 112 137 U+308C 201 3423 181 137 U+3060 243 2830 159 143 U+5F37 1479 2034 163 145 U+306E 1497 1567 123 152 U+305F 1164 952 145 153 U+3066 552 1199 97 155 U+4FF3 537 2095 176 155 U+6839 203 1439 184 156 U+304B 1188 2606 156 157 U+8AE7 549 2328 156 159 U+308C 1495 2784 168 159 U+5B50 891 1255 100 164 U+3092 584 2546 117 164 U+53CA 849 1588 151 164 U+8005 1192 2198 133 169 U+305A 889 1763 103 171 U+907F 513 945 181 171 U+6B63 539 1439 136 172 U+6587 192 2382 216 17

In [2]:
df1['image_id'] = df1.image_id + '.jpg'
df1.head()

,image_id,labels
0,100241706_00004_2.jpg,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...
1,100241706_00005_1.jpg,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...
2,100241706_00005_2.jpg,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...
3,100241706_00006_1.jpg,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...
4,100241706_00007_2.jpg,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...


In [3]:
df1['labels1'] = df1['labels'].str.split(' ')
df1.head()

,image_id,labels,labels1
0,100241706_00004_2.jpg,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...,"[U+306F, 1231, 3465, 133, 53, U+304C, 275, 165..."
1,100241706_00005_1.jpg,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...,"[U+306F, 1087, 2018, 103, 65, U+304B, 1456, 18..."
2,100241706_00005_2.jpg,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...,"[U+306F, 572, 1376, 125, 57, U+306E, 1551, 208..."
3,100241706_00006_1.jpg,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...,"[U+3082, 1455, 3009, 65, 44, U+516B, 1654, 152..."
4,100241706_00007_2.jpg,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...,"[U+309D, 1201, 2949, 27, 33, U+309D, 1196, 153..."


In [4]:
df1 = df1.dropna()
df1 = df1.reset_index(drop=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3605 entries, 0 to 3604
Data columns (total 3 columns):
image_id    3605 non-null object
labels      3605 non-null object
labels1     3605 non-null object
dtypes: object(3)
memory usage: 84.6+ KB


In [0]:
unicode_trans = pd.read_csv('unicode_translation.csv')
unicode_map = {codepoint: char for codepoint, char in unicode_trans.values}
unicode2labels = dict(zip(unicode_map.keys(), range(len(unicode_map.keys()))))

In [0]:
import numpy as np

clz = []
for idx, val in enumerate(df1.labels1):
  elm = val[::5]
  labels = np.zeros(len(elm))
  for i, x in enumerate(elm):
    labels[i] = unicode2labels[x]
  
  clz.append(labels)

In [7]:
np.array(clz).shape

(3605,)

In [0]:
def bb_hw_pandas(a):
  a = np.array(a.split(' ')).reshape(-1, 5)
  b = []
  for codepoint, x, y, w, h in a:
    x, y, w, h = int(x), int(y), int(w), int(h)
    w = x+w
    h = y+h
    z = [x, y, w, h]
    b.append(z)
    
  return b

In [9]:
df1.head()

,image_id,labels,labels1
0,100241706_00004_2.jpg,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...,"[U+306F, 1231, 3465, 133, 53, U+304C, 275, 165..."
1,100241706_00005_1.jpg,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...,"[U+306F, 1087, 2018, 103, 65, U+304B, 1456, 18..."
2,100241706_00005_2.jpg,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...,"[U+306F, 572, 1376, 125, 57, U+306E, 1551, 208..."
3,100241706_00006_1.jpg,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...,"[U+3082, 1455, 3009, 65, 44, U+516B, 1654, 152..."
4,100241706_00007_2.jpg,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...,"[U+309D, 1201, 2949, 27, 33, U+309D, 1196, 153..."


In [10]:
df1['bbox'] = df1['labels'].apply(lambda x : bb_hw_pandas(x))
df1.head()

,image_id,labels,labels1,bbox
0,100241706_00004_2.jpg,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...,"[U+306F, 1231, 3465, 133, 53, U+304C, 275, 165...","[[1231, 3465, 1364, 3518], [275, 1652, 359, 17..."
1,100241706_00005_1.jpg,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...,"[U+306F, 1087, 2018, 103, 65, U+304B, 1456, 18...","[[1087, 2018, 1190, 2083], [1456, 1832, 1496, ..."
2,100241706_00005_2.jpg,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...,"[U+306F, 572, 1376, 125, 57, U+306E, 1551, 208...","[[572, 1376, 697, 1433], [1551, 2080, 1620, 21..."
3,100241706_00006_1.jpg,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...,"[U+3082, 1455, 3009, 65, 44, U+516B, 1654, 152...","[[1455, 3009, 1520, 3053], [1654, 1528, 1795, ..."
4,100241706_00007_2.jpg,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...,"[U+309D, 1201, 2949, 27, 33, U+309D, 1196, 153...","[[1201, 2949, 1228, 2982], [1196, 1539, 1223, ..."


In [11]:
df1.bbox[0][0]

[1231, 3465, 1364, 3518]

In [12]:
import numpy as np
df1['classes'] = np.array(clz)
df1.head()

,image_id,labels,labels1,bbox,classes
0,100241706_00004_2.jpg,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...,"[U+306F, 1231, 3465, 133, 53, U+304C, 275, 165...","[[1231, 3465, 1364, 3518], [275, 1652, 359, 17...","[92.0, 57.0, 52.0, 62.0, 88.0, 88.0, 91.0, 125..."
1,100241706_00005_1.jpg,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...,"[U+306F, 1087, 2018, 103, 65, U+304B, 1456, 18...","[[1087, 2018, 1190, 2083], [1456, 1832, 1496, ...","[92.0, 56.0, 56.0, 52.0, 88.0, 88.0, 101.0, 57..."
2,100241706_00005_2.jpg,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...,"[U+306F, 572, 1376, 125, 57, U+306E, 1551, 208...","[[572, 1376, 697, 1433], [1551, 2080, 1620, 21...","[92.0, 91.0, 101.0, 57.0, 88.0, 110.0, 56.0, 8..."
3,100241706_00006_1.jpg,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...,"[U+3082, 1455, 3009, 65, 44, U+516B, 1654, 152...","[[1455, 3009, 1520, 3053], [1654, 1528, 1795, ...","[111.0, 502.0, 129.0, 88.0, 120.0, 56.0, 91.0,..."
4,100241706_00007_2.jpg,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...,"[U+309D, 1201, 2949, 27, 33, U+309D, 1196, 153...","[[1201, 2949, 1228, 2982], [1196, 1539, 1223, ...","[128.0, 128.0, 128.0, 128.0, 101.0, 128.0, 26...."


In [13]:
df1.classes[0][0]

92.0

In [14]:
columns = ['labels', 'labels1']
df1.drop(columns, inplace=True, axis=1)
df1.head()

,image_id,bbox,classes
0,100241706_00004_2.jpg,"[[1231, 3465, 1364, 3518], [275, 1652, 359, 17...","[92.0, 57.0, 52.0, 62.0, 88.0, 88.0, 91.0, 125..."
1,100241706_00005_1.jpg,"[[1087, 2018, 1190, 2083], [1456, 1832, 1496, ...","[92.0, 56.0, 56.0, 52.0, 88.0, 88.0, 101.0, 57..."
2,100241706_00005_2.jpg,"[[572, 1376, 697, 1433], [1551, 2080, 1620, 21...","[92.0, 91.0, 101.0, 57.0, 88.0, 110.0, 56.0, 8..."
3,100241706_00006_1.jpg,"[[1455, 3009, 1520, 3053], [1654, 1528, 1795, ...","[111.0, 502.0, 129.0, 88.0, 120.0, 56.0, 91.0,..."
4,100241706_00007_2.jpg,"[[1201, 2949, 1228, 2982], [1196, 1539, 1223, ...","[128.0, 128.0, 128.0, 128.0, 101.0, 128.0, 26...."


In [0]:
# Defining the dataset:
'''
https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
'''
import os
import numpy as np
import torch.utils.data
from PIL import Image
import pandas as pd

class KuzushijiDataset(torch.utils.data.Dataset):
  def __init__(self, df, root, mode='train', transforms=None):
    self.records = df.to_records(index=False)
    self.root = root
    self.mode = mode
    self.transforms = transforms
    self.len = df.shape[0]    
    
  def __getitem__(self, index):
    imgs, bbox, clas = self.records[index].image_id, self.records[index].bbox, self.records[index].classes
    img = Image.open(os.path.join(root, imgs)).convert("RGB")
    
    target = {}
    target["boxes"] = torch.as_tensor(bbox, dtype=torch.float32)
    target["labels"] = torch.as_tensor(clas, dtype=torch.int64)
    target["image_id"] = torch.tensor([index])
    
    if self.transforms is not None:
      img = self.transforms(img)
      #img, target = self.transforms(img, target)
      
    return img, target
  
    if self.mode == 'train':
      return img, target
    else:
      return img, self.records[index].image_id
    
  def __len__(self):
    return self.len

In [16]:
import matplotlib.pyplot as plt

root = 'train_images'

dataset = KuzushijiDataset(df1, root)

for i in range(len(dataset)):
  img, lbl = dataset[i]
  
  print(i, img.size, len(lbl))
  
  if i == 2:
    break

0 (2404, 3874) 3
1 (2392, 3874) 3
2 (2416, 3874) 3


In [17]:
dataset[0]

(<PIL.Image.Image image mode=RGB size=2404x3874 at 0x7F443E413CF8>,
 {'boxes': tensor([[1231., 3465., 1364., 3518.],
          [ 275., 1652.,  359., 1721.],
          [1495., 1218., 1638., 1287.],
          [ 220., 3331.,  273., 3422.],
          [ 911., 1452.,  972., 1544.],
          [ 927., 3445.,  998., 3537.],
          [ 904., 2879.,  999., 2971.],
          [1168., 1396., 1355., 1491.],
          [ 289., 3166.,  358., 3263.],
          [ 897., 3034., 1018., 3141.],
          [ 547., 1912.,  688., 2020.],
          [1489., 2675., 1640., 2784.],
          [1561., 2979., 1616., 3095.],
          [1513., 2500., 1640., 2617.],
          [1213., 1523., 1285., 1642.],
          [1219., 3266., 1314., 3390.],
          [ 259., 2230.,  327., 2355.],
          [1184., 2423., 1353., 2548.],
          [ 849., 2236., 1012., 2363.],
          [1144., 1212., 1344., 1340.],
          [ 316., 3287.,  373., 3420.],
          [ 217., 2044.,  400., 2179.],
          [ 277., 2974.,  389., 3111.],
   

In [18]:
%%writefile utils.py
from __future__ import print_function

from collections import defaultdict, deque
import datetime
import pickle
import time

import torch
import torch.distributed as dist

import errno
import os


class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)


def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]

    # serialized to a Tensor
    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    # obtain Tensor size of each rank
    local_size = torch.tensor([tensor.numel()], device="cuda")
    size_list = [torch.tensor([0], device="cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    # receiving Tensor from all ranks
    # we pad the tensor because torch all_gather does not support
    # gathering tensors of different shapes
    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.empty((max_size,), dtype=torch.uint8, device="cuda"))
    if local_size != max_size:
        padding = torch.empty(size=(max_size - local_size,), dtype=torch.uint8, device="cuda")
        tensor = torch.cat((tensor, padding), dim=0)
    dist.all_gather(tensor_list, tensor)

    data_list = []
    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        log_msg = self.delimiter.join([
            header,
            '[{0' + space_fmt + '}/{1}]',
            'eta: {eta}',
            '{meters}',
            'time: {time}',
            'data: {data}',
            'max mem: {memory:.0f}'
        ])
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                print(log_msg.format(
                    i, len(iterable), eta=eta_string,
                    meters=str(self),
                    time=str(iter_time), data=str(data_time),
                    memory=torch.cuda.max_memory_allocated() / MB))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))


def collate_fn(batch):
    return tuple(zip(*batch))


def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):

    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    elif 'SLURM_PROCID' in os.environ:
        args.rank = int(os.environ['SLURM_PROCID'])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print('Not using distributed mode')
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}'.format(
        args.rank, args.dist_url), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank)
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

Overwriting utils.py


In [19]:
import torchvision.transforms as T

root = 'train_images'

dataset = KuzushijiDataset(df1, root, transforms = T.ToTensor())
#dataset_test = KuzushijiDataset(df1, root, transforms = T.ToTensor())

# split the dataset in train and test set
torch.manual_seed(1)
#indices = torch.randperm(len(dataset)).tolist()
#dataset = torch.utils.data.Subset(dataset, indices[:-50])
#dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

In [20]:
for i in range(len(dataset)):
  img, lbl = dataset[i]
  
  print(i, img.shape, len(lbl))
  
  if i == 2:
    break

0 torch.Size([3, 3874, 2404]) 3
1 torch.Size([3, 3874, 2392]) 3
2 torch.Size([3, 3874, 2416]) 3


In [0]:
# define training and validation data loaders

#from engine import train_one_epoch, evaluate
import utils

data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,
                                          shuffle=True,
                                          #num_workers=4,
                                          collate_fn=utils.collate_fn)

#data_loader_test = torch.utils.data.DataLoader(dataset_test,
 #                                              batch_size=1,
  #                                             shuffle=False,
                                               #num_workers=4,
   #                                            collate_fn=utils.collate_fn)

In [0]:
dataiter = iter(data_loader)
images, labels = dataiter.next()

In [23]:
len(images)

1

In [24]:
len(labels)

1

In [25]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 4788  # 4212 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 93.7MB/s]


In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [27]:
images = list(image.to(device) for image in images)
labels = [{k: v.to(device) for k, v in t.items()} for t in labels]
output = model(images, labels)
output

{'loss_box_reg': tensor(0.2220, device='cuda:0', grad_fn=<DivBackward0>),
 'loss_classifier': tensor(8.3762, device='cuda:0', grad_fn=<NllLossBackward>),
 'loss_objectness': tensor(4.2284, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(0.4006, device='cuda:0', grad_fn=<DivBackward0>)}

In [28]:
from engine import train_one_epoch, evaluate

'''
ERROR
'''
# let's train it for 10 epochs
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    #evaluate(model, data_loader_test, device=device)

Epoch: [0]  [   0/3605]  eta: 1:19:35  lr: 0.000010  loss: 13.5540 (13.5540)  loss_classifier: 8.3091 (8.3091)  loss_box_reg: 0.3930 (0.3930)  loss_objectness: 4.5431 (4.5431)  loss_rpn_box_reg: 0.3089 (0.3089)  time: 1.3246  data: 0.3370  max mem: 2963
Epoch: [0]  [  10/3605]  eta: 1:07:46  lr: 0.000060  loss: 11.9608 (11.5051)  loss_classifier: 8.1953 (8.1991)  loss_box_reg: 0.3930 (0.3544)  loss_objectness: 2.9997 (2.6629)  loss_rpn_box_reg: 0.2988 (0.2888)  time: 1.1313  data: 0.2200  max mem: 4413
Epoch: [0]  [  20/3605]  eta: 1:06:16  lr: 0.000110  loss: 9.1524 (10.0768)  loss_classifier: 8.0683 (7.8679)  loss_box_reg: 0.3760 (0.3681)  loss_objectness: 0.4545 (1.5555)  loss_rpn_box_reg: 0.2725 (0.2853)  time: 1.0983  data: 0.2068  max mem: 5580
Epoch: [0]  [  30/3605]  eta: 1:08:00  lr: 0.000160  loss: 7.3433 (8.6173)  loss_classifier: 6.4991 (6.8083)  loss_box_reg: 0.3855 (0.3828)  loss_objectness: 0.2669 (1.1316)  loss_rpn_box_reg: 0.2636 (0.2946)  time: 1.1472  data: 0.2122  m

In [0]:
import utils
import math
import sys

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

In [0]:
epoch = 1
print('Epoch: [{}]'.format(epoch))
train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

In [0]:
# test dataset:
# Defining the dataset:
'''
https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
'''
import os
import numpy as np
import torch.utils.data
from PIL import Image
import pandas as pd

class KuzushijiTDataset(torch.utils.data.Dataset):
  def __init__(self, df, root, mode='train', transforms=None):
    self.records = df.to_records(index=False)
    self.root = root
    self.mode = mode
    self.transforms = transforms
    self.len = df.shape[0]    
    
  def __getitem__(self, index):
    imgs = self.records[index].image_id
    img = Image.open(os.path.join(root, imgs)).convert("RGB")
    
    img_id = torch.tensor([index])
    
    if self.transforms is not None:
      img = self.transforms(img)
      
    return img, img_id
    
  def __len__(self):
    return self.len

In [30]:
import pandas as pd
submission = pd.read_csv('sample_submission.csv')
print(submission.shape, submission.head())

(4150, 2)         image_id                 labels
0  test_00145af3  U+003F 1 1 U+FF2F 2 2
1  test_001c37e2  U+003F 1 1 U+FF2F 2 2
2  test_003aa33a  U+003F 1 1 U+FF2F 2 2
3  test_00665e33  U+003F 1 1 U+FF2F 2 2
4  test_006964dc  U+003F 1 1 U+FF2F 2 2


In [31]:
submission['image_id'] = submission.image_id + '.jpg'
submission.head()

,image_id,labels
0,test_00145af3.jpg,U+003F 1 1 U+FF2F 2 2
1,test_001c37e2.jpg,U+003F 1 1 U+FF2F 2 2
2,test_003aa33a.jpg,U+003F 1 1 U+FF2F 2 2
3,test_00665e33.jpg,U+003F 1 1 U+FF2F 2 2
4,test_006964dc.jpg,U+003F 1 1 U+FF2F 2 2


In [32]:
submission.shape

(4150, 2)

In [33]:
import matplotlib.pyplot as plt

root = 'test_images'

Tdataset = KuzushijiTDataset(submission, root, transforms=T.ToTensor())

for i in range(len(dataset)):
  img, lbl = Tdataset[i]
  
  print(i, img.shape, lbl.shape)
  
  if i == 2:
    break

0 torch.Size([3, 3394, 2210]) torch.Size([1])
1 torch.Size([3, 3134, 2046]) torch.Size([1])
2 torch.Size([3, 3122, 1990]) torch.Size([1])


In [34]:
Tdataset[0]

(tensor([[[0.5882, 0.5961, 0.6039,  ..., 0.6980, 0.7020, 0.7020],
          [0.5882, 0.5922, 0.6039,  ..., 0.6941, 0.7020, 0.7020],
          [0.5843, 0.5882, 0.6000,  ..., 0.6941, 0.6980, 0.6980],
          ...,
          [0.5843, 0.5961, 0.6000,  ..., 0.6824, 0.6824, 0.6824],
          [0.5804, 0.5922, 0.6039,  ..., 0.6863, 0.6902, 0.6902],
          [0.5882, 0.6039, 0.6118,  ..., 0.6863, 0.6902, 0.6902]],
 
         [[0.5686, 0.5765, 0.5843,  ..., 0.6980, 0.7020, 0.7020],
          [0.5686, 0.5725, 0.5843,  ..., 0.6941, 0.7020, 0.7020],
          [0.5647, 0.5686, 0.5804,  ..., 0.6941, 0.6980, 0.6980],
          ...,
          [0.5647, 0.5765, 0.5804,  ..., 0.6784, 0.6784, 0.6784],
          [0.5608, 0.5725, 0.5843,  ..., 0.6824, 0.6863, 0.6863],
          [0.5686, 0.5843, 0.5922,  ..., 0.6824, 0.6863, 0.6863]],
 
         [[0.5451, 0.5529, 0.5608,  ..., 0.6667, 0.6706, 0.6706],
          [0.5451, 0.5490, 0.5608,  ..., 0.6627, 0.6706, 0.6706],
          [0.5412, 0.5451, 0.5569,  ...,

In [0]:
import utils

Tdata_loader = torch.utils.data.DataLoader(Tdataset,
                                          batch_size=1,
                                          shuffle=False,
                                          #num_workers=4,
                                          collate_fn=utils.collate_fn)

In [0]:
for images, labels in Tdata_loader:
  print(labels)

In [0]:
pred = []
with torch.no_grad():
  model.eval()
  for images, labels in Tdata_loader:
    images = list(image.to(device) for image in images)
    output = model(images)
    res = {labels: output}
    #labels = [{k: v.to(device) for k, v in t.items()} for t in labels]
   # output = model(images, labels)
    #res = {label["image_id"].item(): outputs for label, outputs in zip(labels, output)}
    pred.append(res)
    del images, output, res

In [37]:
len(pred)

4150

In [56]:
a = pred[0]
for k, v in a.items():
  print(int(k[0].cpu().numpy()))

0


In [53]:
a = pred[0]
for k, v in a.items():
  b = v[0]
  c = b['boxes'].cpu().numpy()
  print(c.shape)

(100, 4)


In [58]:
records = submission.to_records(index=False)
records[0]

('test_00145af3.jpg', 'U+003F 1 1 U+FF2F 2 2')

In [0]:
labels2unicode = {v: k for k, v in unicode2labels.items()}
labels2unicode

In [0]:
comps = []
img_id = []
for elm in pred:
  for k, v in elm.items():
    
    img_id.append(records[int(k[0].cpu().numpy())].image_id)
    b = v[0]
    box = b['boxes'].cpu().numpy()
    lb = b['labels'].cpu().numpy()
    
    ulb = []
    
    for em in lb:
      uelm = labels2unicode[em]
      ulb.append(uelm)
      
    comp = []
    
    for i in range(lb.shape[0]):
      sng = [ulb[i], box[i]]
      comp.append(sng)
      #comp.append(ulb[i])
      #comp.append(box[i])
    
    comps.append(comp)

In [60]:
len(comps[0])

100

In [98]:
comps[0]

[['U+3075',
  array([ 998.8686, 1548.1893, 1121.5999, 1639.9849], dtype=float32)],
 ['U+3084',
  array([ 662.05084, 2510.2322 ,  782.7215 , 2587.9036 ], dtype=float32)],
 ['U+3075',
  array([ 498.54675, 2867.594  ,  607.64044, 2974.871  ], dtype=float32)],
 ['U+3084',
  array([1668.7532,  953.1088, 1781.6333, 1045.1973], dtype=float32)],
 ['U+3066',
  array([ 532.74506, 1693.211  ,  615.25037, 1789.0995 ], dtype=float32)],
 ['U+306E',
  array([1337.6643, 1930.4438, 1425.0494, 2023.9517], dtype=float32)],
 ['U+3066',
  array([1369.6523,  824.2311, 1446.9543,  929.3372], dtype=float32)],
 ['U+3084',
  array([ 824.80505, 2301.4597 ,  947.4589 , 2379.9485 ], dtype=float32)],
 ['U+3066',
  array([1500.4342, 2362.0554, 1576.6232, 2455.369 ], dtype=float32)],
 ['U+3044',
  array([1016.46344, 1480.9626 , 1107.9977 , 1544.611  ], dtype=float32)],
 ['U+306E',
  array([ 207.61516, 1418.5038 ,  309.52695, 1506.055  ], dtype=float32)],
 ['U+3075',
  array([ 505.91068, 1896.2976 ,  615.3127 , 1982.6

In [0]:
d = []
for item in comps:
  c = [] 
  for i in item:
    a = i[1]
    x, y, z, w = int(a[0]), int(a[1]), int(a[2]) - int(a[0]), int(a[3]) - int(a[1])
    #b = [i[0], x, y, z, w]
    c.append(i[0])
    c.append(x)
    c.append(y)
    c.append(z)
    c.append(w)
  d.append(np.asarray(c))

In [100]:
d[0].shape

(500,)

In [101]:
e[0].shape

(500,)

In [102]:
e = np.array([np.array(xi) for xi in d])
e

array([array(['U+3075', '998', '1548', '123', '91', 'U+3084', '662', '2510',
       '120', '77', 'U+3075', '498', '2867', '109', '107', 'U+3084',
       '1668', '953', '113', '92', 'U+3066', '532', '1693', '83', '96',
       'U+306E', '1337', '1930', '88', '93', 'U+3066', '1369', '824',
       '77', '105', 'U+3084', '824', '2301', '123', '78', 'U+3066',
       '1500', '2362', '76', '93', 'U+3044', '1016', '1480', '91', '64',
       'U+306E', '207', '1418', '102', '88', 'U+3075', '505', '1896',
       '110', '86', 'U+3066', '714', '673', '79', '89', 'U+3075', '1794',
       '1553', '118', '90', 'U+3046', '866', '2385', '40', '133',
       'U+3066', '1207', '860', '60', '117', 'U+3066', '183', '2513',
       '76', '105', 'U+308C', '532', '653', '128', '101', 'U+3075', '174',
       '2087', '113', '99', 'U+3064', '541', '571', '93', '75', 'U+3044',
       '195', '1833', '102', '69', 'U+304B', '1155', '2490', '99', '56',
       'U+3066', '1697', '1325', '44', '132', 'U+306B', '1512', '1267

In [0]:
columns = ['image_id', 'labels']
pddf = pd.DataFrame(index = range(len(img_id)), columns = columns)

In [0]:
pddf['image_id'] = img_id
pddf['labels'] =np.array(e)#pd.Series(e)

In [105]:
pddf.head()

,image_id,labels
0,test_00145af3.jpg,"[U+3075, 998, 1548, 123, 91, U+3084, 662, 2510..."
1,test_001c37e2.jpg,"[U+3057, 1290, 1824, 86, 99, U+306E, 1143, 140..."
2,test_003aa33a.jpg,"[U+306E, 1276, 1601, 61, 65, U+306E, 1283, 236..."
3,test_00665e33.jpg,"[U+3068, 476, 2096, 31, 98, U+306E, 584, 503, ..."
4,test_006964dc.jpg,"[U+3084, 1673, 2775, 127, 84, U+3084, 890, 218..."


In [106]:
pddf['image_id'] = pddf['image_id'].map(lambda x: x.rstrip('.jpg'))
pddf.head()

,image_id,labels
0,test_00145af3,"[U+3075, 998, 1548, 123, 91, U+3084, 662, 2510..."
1,test_001c37e2,"[U+3057, 1290, 1824, 86, 99, U+306E, 1143, 140..."
2,test_003aa33a,"[U+306E, 1276, 1601, 61, 65, U+306E, 1283, 236..."
3,test_00665e33,"[U+3068, 476, 2096, 31, 98, U+306E, 584, 503, ..."
4,test_006964dc,"[U+3084, 1673, 2775, 127, 84, U+3084, 890, 218..."


In [107]:
pddf['labels'] = [','.join(map(str, l)) for l in pddf['labels']]
pddf.head()

,image_id,labels
0,test_00145af3,"U+3075,998,1548,123,91,U+3084,662,2510,120,77,..."
1,test_001c37e2,"U+3057,1290,1824,86,99,U+306E,1143,1404,67,71,..."
2,test_003aa33a,"U+306E,1276,1601,61,65,U+306E,1283,2367,60,63,..."
3,test_00665e33,"U+3068,476,2096,31,98,U+306E,584,503,95,105,U+..."
4,test_006964dc,"U+3084,1673,2775,127,84,U+3084,890,2180,124,97..."


In [108]:
pddf['labels'] = pddf['labels'].str.replace(',', ' ')
pddf.head()

,image_id,labels
0,test_00145af3,U+3075 998 1548 123 91 U+3084 662 2510 120 77 ...
1,test_001c37e2,U+3057 1290 1824 86 99 U+306E 1143 1404 67 71 ...
2,test_003aa33a,U+306E 1276 1601 61 65 U+306E 1283 2367 60 63 ...
3,test_00665e33,U+3068 476 2096 31 98 U+306E 584 503 95 105 U+...
4,test_006964dc,U+3084 1673 2775 127 84 U+3084 890 2180 124 97...


In [0]:
pddf.to_csv('submission1.csv', index=False)

In [94]:
pddf['labels'][0]

'U+3075 998 1548 1121 1639 U+3084 662 2510 782 2587 U+3075 498 2867 607 2974 U+3084 1668 953 1781 1045 U+3066 532 1693 615 1789 U+306E 1337 1930 1425 2023 U+3066 1369 824 1446 929 U+3084 824 2301 947 2379 U+3066 1500 2362 1576 2455 U+3044 1016 1480 1107 1544 U+306E 207 1418 309 1506 U+3075 505 1896 615 1982 U+3066 714 673 793 762 U+3075 1794 1553 1912 1643 U+3046 866 2385 906 2518 U+3066 1207 860 1267 977 U+3066 183 2513 259 2618 U+308C 532 653 660 754 U+3075 174 2087 287 2186 U+3064 541 571 634 646 U+3044 195 1833 297 1902 U+304B 1155 2490 1254 2546 U+3066 1697 1325 1741 1457 U+306B 1512 1267 1610 1365 U+3042 1803 2702 1893 2801 U+3092 1031 894 1145 1011 U+3068 56 2551 84 2666 U+306B 80 1134 125 1230 U+3046 878 1867 918 2001 U+308C 1821 969 1937 1075 U+3046 571 924 607 1038 U+304D 729 1165 783 1283 U+3068 422 1008 453 1112 U+3068 1060 1339 1091 1458 U+3068 1700 1573 1730 1686 U+3075 987 2202 1098 2303 U+3066 1212 1909 1254 2057 U+3066 1852 1075 1921 1159 U+3068 100 1547 129 1663 U+306